<a href="https://colab.research.google.com/github/lbrandoli/ProgramacionConcurrente/blob/main/TP_1_Parte_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **C++**

Codigo del arbol de procesos.

In [ ]:
%%writefile arbol.cpp
#include <iostream>
#include <unistd.h>
#include <sys/wait.h>

using namespace std;

void mostrarProceso(char proceso)
{
   cout<<"Proceso "<<proceso<<" --> "<<"PID:"<<getpid()<<"  "<<"PPID:"<<getppid()<<endl;
}

int main(int argc, char*argv[])
{
  pid_t pidA,pidB,pidC,pidD,pidE,pidF,pidG,pidH;
  pidA=getpid();
  mostrarProceso('A');
  pidB=fork();
  if(pidB == 0)
  {
    pidB=getpid();
    mostrarProceso('B');
    pidD=fork();
    if(pidD == 0)
    {
      pidD=getpid();
      mostrarProceso('D');
      sleep(30);
      exit(0);
    }
    pidE=fork();
    if(pidE == 0)
    {
      pidE=getpid();
      mostrarProceso('E');
      pidG=fork();
      if(pidG == 0)
      {
        pidG=getpid();
        mostrarProceso('G');
        sleep(30);
        exit(0);
      }
       pidH=fork();
      if(pidH == 0)
      {
        pidH=getpid();
        mostrarProceso('H');
        sleep(30);
        exit(0);
      }
      sleep(30);
      wait(NULL);
      wait(NULL);
      exit(0);
    }
    sleep(30);
    wait(NULL);
    wait(NULL);
    exit(0);
  }
  pidC=fork();
  if(pidC == 0)
  {
    pidC=getpid();
    mostrarProceso('C');
    pidF=fork();
    if(pidF == 0)
    {
      pidF=getpid();
      mostrarProceso('F');
      sleep(30);
      exit(0);
    }
    sleep(30);
    wait(NULL);
    exit(0);
  }
  sleep(30);
  wait(NULL);
  wait(NULL);
  return 0;
}

Overwriting arbol.cpp


2-Verificar que existe el archivo arbol.cpp

In [ ]:
!ls -l

total 32
-rwxr-xr-x 1 root root 16944 Sep 10 23:48 arbol.bin
-rw-r--r-- 1 root root  1365 Sep 10 23:59 arbol.cpp
-rw-r--r-- 1 root root   269 Sep 10 23:52 resultados
drwxr-xr-x 1 root root  4096 Sep  7 13:24 sample_data


3-Compilar el archivo arbol.cpp y generar el binario.

In [ ]:
%%shell
g++ arbol.cpp -o arbol.bin

4-Ejecutar el programa y mostrar la salida en el archivo "resultados".

In [ ]:
%%shell
nohup ./arbol.bin 1> resultados 2> /dev/null &

5-Observar los procesos generados por este programa.

In [ ]:
%%shell
ps -ef | grep arbol.bin

root        4502       1  0 23:59 ?        00:00:00 ./arbol.bin
root        4503    4502  0 23:59 ?        00:00:00 ./arbol.bin
root        4504    4502  0 23:59 ?        00:00:00 ./arbol.bin
root        4505    4504  0 23:59 ?        00:00:00 ./arbol.bin
root        4506    4503  0 23:59 ?        00:00:00 ./arbol.bin
root        4507    4503  0 23:59 ?        00:00:00 ./arbol.bin
root        4508    4507  0 23:59 ?        00:00:00 ./arbol.bin
root        4509    4507  0 23:59 ?        00:00:00 ./arbol.bin
root        4522     875  0 23:59 ?        00:00:00 /bin/bash -c ps -ef | grep arbol.bin 
root        4524    4522  0 23:59 ?        00:00:00 grep arbol.bin


6-Observar el arbol de procesos.

In [ ]:
%%shell
pstree -pc 4502

arbol.bin(4502)─┬─arbol.bin(4503)─┬─arbol.bin(4506)
                │                 └─arbol.bin(4507)─┬─arbol.bin(4508)
                │                                   └─arbol.bin(4509)
                └─arbol.bin(4504)───arbol.bin(4505)


7-Observar la salida en el archivo "resultados".

In [ ]:
!cat resultados

Proceso A --> PID:4502  PPID:1
Proceso C --> PID:4504  PPID:4502
Proceso F --> PID:4505  PPID:4504
Proceso B --> PID:4503  PPID:4502
Proceso E --> PID:4507  PPID:4503
Proceso H --> PID:4509  PPID:4507
Proceso D --> PID:4506  PPID:4503
Proceso G --> PID:4508  PPID:4507


# **Python**

In [ ]:
%%writefile tp1_procesos.py
import os
import time

PROCESO_A = "A"
PROCESO_B = "B"
PROCESO_C = "C"
PROCESO_D = "D"
PROCESO_E = "E"
PROCESO_F = "F"
PROCESO_G = "G"
PROCESO_H = "H"
TIME_SLEEP = 200
PID_NULL = 0

def processMessage(node):
    message = "Hola mi nombre es {name}. PID: {pid}. PPID {ppid}\n"
    print(message.format(name=node,pid=os.getpid(),ppid=os.getppid()))

def spawnProcesses(node):
    pid=PID_NULL
    name,children = node
    processMessage(name)
    for item in children:
        pid=os.fork()
        if pid < PID_NULL:
            print("Error al crear nuevo proceso")
            os._exit(1)
        if not pid:
            spawnProcesses(item)
            time.sleep(TIME_SLEEP)
            os._exit(0)
    for item in children:
        os.wait()

def main():
    tree = (PROCESO_A, [(PROCESO_B,[(PROCESO_D,[]), (PROCESO_E,[(PROCESO_G,[]), (PROCESO_H,[])])]),(PROCESO_C,[(PROCESO_F,[])])])
    spawnProcesses(tree)
    os._exit(0)

main()

Overwriting tp1_procesos.py


In [ ]:
!touch resultado_py && chmod 700 resultado_py

In [ ]:
!nohup python3 -u tp1.py 1>resultado_py 2>/dev/null &

In [ ]:
!cat resultado_py

In [ ]:
!ps -ef | grep tp1_procesos.py

In [ ]:
!pstree -pc $(pgrep -f tp1_procesos.py -P 1)

#**Java**

In [ ]:
%%writefile ArbolDeProcesos.java
import java.io.IOException;

public class ArbolDeProcesos {
    static int cantHijosA = 2;
    static int cantHijosB = 2;
    static int cantHijosC = 1;
    static int cantHijosE = 2;

    public static void main(String[] args)
    {
        int cantHijos = 0;
        char nombreNodo = 'A';
        Process procesos[] = null;
        String procesoActual = "A";
        if (args.length != 0)
            procesoActual = args[0];

        ProcessHandle obtenerPID = ProcessHandle.current();

        System.out.println("soy el proceso " + procesoActual + " de PID: " + obtenerPID.pid() + " y PPID: "
                + obtenerPID.parent().get().pid());

        if (procesoActual.equals("A"))
        {
            cantHijos = cantHijosA;
            procesos = new Process[cantHijosA];
        }
        else if (procesoActual.equals("B"))
        {
            cantHijos = cantHijosB;
            procesos = new Process[cantHijosB];
            nombreNodo += cantHijosA;
        }
        else if (procesoActual.equals("C"))
        {
            cantHijos = cantHijosC;
            procesos = new Process[cantHijosC];
            nombreNodo = (char) ('C' + cantHijosB);
        }
        else if (procesoActual.equals("E"))
        {
            cantHijos = cantHijosE;
            procesos = new Process[cantHijosE];
            nombreNodo = (char) ('E' + cantHijosC);
        }
        else
        {
            try
            {
                Thread.sleep(10000);
            }
            catch (InterruptedException e)
            {
                e.printStackTrace();
            }
            return;
        }

        for (int i = 1; i <= cantHijos; i++)
        {
            ProcessBuilder proceso = new ProcessBuilder("java", "ArbolDeProcesos.java",
                    Character.toString(nombreNodo + i));
            proceso.inheritIO();
            try
            {
                procesos[i - 1] = proceso.start();
            }
            catch (IOException e)
            {
                e.printStackTrace();
            }
        }
        for (int i = 0; i < cantHijos; i++)
        {
            try
            {
                procesos[i].waitFor();
            }
             catch (InterruptedException e)
            {
                e.printStackTrace();
            }
        }
    }
}


Overwriting ArbolDeProcesos.java


In [ ]:
!java ArbolDeProcesos.java

In [ ]:
!nohup java ArbolDeProcesos.java 1>salida 2>/dev/null &

In [ ]:
!ps -ef | grep ArbolDeProcesos.java

In [ ]:
!pstree -pT 7960

java(7960)─┬─java(7982)─┬─java(8039)
           │            └─java(8048)─┬─java(8112)
           │                         └─java(8115)
           └─java(7984)───java(8037)
